In [1]:
import sys
sys.path.append('..')
from functions.time_domain import *
from functions.frequency_domain import *

In [2]:
percent_frame_size = 0.15
percent_hop_length = 0.5
path = '../recordings/4_10/Znormalizowane/bapis_1.wav'
audio, frame_rate, audio_time, n_samples  = read_wave(path)
frames, n_, N_= split_to_frames(audio, frame_rate, percent_frame_size=percent_frame_size, percent_hop_length=percent_hop_length)
fft_frames = transform_frames_to_frequency_domain(frames, frame_rate, N_, window_type='hann')

In [3]:
import numpy as np

In [4]:
def preemphasis(signal, coeff=0.97, frame_rate=None, display=False):
    preepmhasized_signal = np.append(signal[0], signal[1:] - coeff * signal[:-1])
    if display:
        displayhook(Audio(preepmhasized_signal, rate=frame_rate))
    return preepmhasized_signal
 

In [5]:
preepmhasized_audio = preemphasis(audio, frame_rate=frame_rate, display=True)

In [6]:
frames, n_, N_ = split_to_frames(preepmhasized_audio, frame_rate, percent_frame_size=percent_frame_size, percent_hop_length=percent_hop_length, set_frame_size=256)

In [7]:
fft_frames = transform_frames_to_frequency_domain(frames, frame_rate, N_, window_type='hamming')

In [8]:
amplitude = np.abs(fft_frames)
magnitude = amplitude**2

In [11]:
import numpy as np
from scipy.fftpack import dct

# Parametry banku filtrów trójkątnych
n_filters = 32  # Liczba filtrów
n_mfcc = 16  # Liczba współczynników MFCC
lower_freq = 0  # Dolna granica pasma częstotliwości (w Hz)
upper_freq = 8000  # Górna granica pasma częstotliwości (w Hz)

# Obliczenie punktów granicznych filtrów trójkątnych w skali mel
mel_lower = 2595 * np.log10(1 + lower_freq / 700)
mel_upper = 2595 * np.log10(1 + upper_freq / 700)
mel_points = np.linspace(mel_lower, mel_upper, n_filters + 2)

# Przekształcenie punktów granicznych do skali częstotliwości
hz_points = 700 * (10**(mel_points / 2595) - 1)

# Obliczenie indeksów dla częstotliwości w skali FFT
n_fft = 2048  # Liczba punktów FFT
fft_freqs = np.linspace(0, frame_rate // 2, n_fft // 2 + 1)

# Znalezienie indeksów dla granic filtrów trójkątnych
filter_indices = np.searchsorted(fft_freqs, hz_points)

# Inicjalizacja macierzy filtrów trójkątnych
filters = np.zeros((n_filters, n_fft // 2 + 1))

# Wypełnienie macierzy filtrami trójkątnymi
for i in range(1, n_filters + 1):
    filters[i-1, filter_indices[i-1]:filter_indices[i]] = \
        (fft_freqs[filter_indices[i-1]:filter_indices[i]] - hz_points[i-1]) / \
        (hz_points[i] - hz_points[i-1])
    filters[i-1, filter_indices[i]:filter_indices[i+1]] = \
        (hz_points[i+1] - fft_freqs[filter_indices[i]:filter_indices[i+1]]) / \
        (hz_points[i+1] - hz_points[i])

# Obliczenie widma mocy dla sygnału audio
fft_spectrum = np.abs(np.fft.rfft(preepmhasized_audio, n_fft))
power_spectrum = fft_spectrum**2

# Przemnożenie widma mocy przez filtry trójkątne
filtered_spectrum = np.dot(filters, power_spectrum)

# Obliczenie logarytmu widma
log_spectrum = np.log10(filtered_spectrum)

# Obliczenie DCT na logarytmie widma
mfcc = dct(log_spectrum, type=2, axis=0, norm='ortho')[:n_mfcc]

# Transpozycja wynikowej macierzy MFCC
mfcc = mfcc.T

In [12]:
mfcc

array([-8.11483744e+00, -1.01933648e+00, -1.25088757e+00, -1.44267693e+00,
        6.38756489e-01,  1.13315726e+00,  1.56839316e-01, -7.16143298e-01,
       -3.97016545e-01,  3.49962449e-01,  5.56565747e-01,  4.62762941e-01,
        2.83808257e-01,  4.08876447e-03,  3.12578930e-01,  4.15512499e-01])